In [1]:
import re
import yaml

In [3]:
with open('src/judge.yml', 'r', encoding="utf-8") as file:
    file_data = file.read()
    data = yaml.safe_load(file_data)

In [13]:
struct = data['protocols'][0]['struct']
struct

'uint8_t game_type : 4;\nuint8_t game_progress : 4; \nuint16_t stage_remain_time;\nuint64_t SyncTimeStamp;'

In [35]:
typeInfoTable = {
    "uint8_t":{"ros":"", "size":1},
    "uint16_t":{"ros":"", "size":2},
    "uint32_t":{"ros":"", "size":4},
    "uint64_t":{"ros":"", "size":8},
    "int":{"ros":"", "size":2},
    "int8_t":{"ros":"", "size":1},
    "int16_t":{"ros":"", "size":2},
    "int32_t":{"ros":"", "size":4},
    "int64_t":{"ros":"", "size":8},
    "float":{"ros":"", "size":4},
    "char":{"ros":"", "size":1},
}

In [68]:
struct = '''uint8_t game_type : 4;
            uint8_t game_progress : 3;
            uint16_t stage_remain_time;
            uint64_t SyncTimeStamp[3];
'''
struct

'uint8_t game_type : 4;\n            uint8_t game_progress : 3;\n            uint16_t stage_remain_time;\n            uint64_t SyncTimeStamp[3];\n'

In [69]:
r = re.compile('\n*?(?P<type>\S*?)\s*?(?P<key>\S*?)\[?(?P<arraysize>\d*?)\]?\s*?:?\s*?(?P<bitsize>\d*?);')
structure = [m.groupdict() for m in r.finditer(struct)]
structure

[{'type': 'uint8_t', 'key': 'game_type', 'arraysize': '', 'bitsize': '4'},
 {'type': 'uint8_t', 'key': 'game_progress', 'arraysize': '', 'bitsize': '3'},
 {'type': 'uint16_t',
  'key': 'stage_remain_time',
  'arraysize': '',
  'bitsize': ''},
 {'type': 'uint64_t', 'key': 'SyncTimeStamp', 'arraysize': '3', 'bitsize': ''}]

In [70]:
size = 0
for item in structure:
    item['typeinfo'] = typeInfo[item['type']]
    size += item['typeinfo']['size']*int(item['arraysize'] or '1')
size

28

In [83]:
a={"aa":1}
b={"b":2}
a.update(b)

In [72]:
a.get("c","")

In [121]:
a.values()

dict_values([1, 2])

In [125]:
a=[1,2,3]
b=[4,5,6]
[*zip(a,b)]

[(1, 4), (2, 5), (3, 6)]

In [5]:
import bitstring

In [150]:
protocol_info_table = {
    'autoaim': {'id': 0x401, 'description':'for test', 'size':24, 'struct':{
        'x':   {'format':'floatle:32'},
        'y':   {'format':'floatle:32'},
        'z':   {'format':'floatle:32', 'arraysize':2},
        'test':{'struct': 'Test_Type', 'arraysize':2}
    }},
    'Test_Type': {'size':4, 'struct':{
        'x':   {'format':'floatle:32'}
    }},
}

In [156]:
def pack_data(name, data):
    formats = []
    data_items = []
    for (key, info) in protocol_info_table[name]['struct'].items():
        if 'arraysize' in info:
            data_list = data[key]
        else:
            data_list = [data[key]]
        for new_data in data_list:
            if 'struct' in info:
                info['format'], new_data = pack_data(info['struct'], new_data)
                data_items += new_data
            else:
                data_items += [new_data]
            formats += [info['format']]
    return (",".join(formats), data_items)

test_data = {'x':1.11,'y':2.22,'z':[3.33,4.44],'test':[{'x':5.55},{'x':6.66}]}
pack_data('autoaim', test_data)

('floatle:32,floatle:32,floatle:32,floatle:32,floatle:32,floatle:32',
 [1.11, 2.22, 3.33, 4.44, 5.55, 6.66])

In [160]:
def pack(name, data, seq=0, skip_crc8=False):
    protocol_info = protocol_info_table[name]
    packet = bitstring.pack('<BHB', *[0xa5, protocol_info['size'], seq])
    if not skip_crc8:
        packet += bytes([helpers.get_crc8_checksum(packet.tobytes())])
    packet += bitstring.pack('<H', protocol_info['id'])
    format_string, data_items = pack_data(name, data)
    packet += bitstring.pack(format_string, *data_items)
    packet += bitstring.pack('<H', helpers.get_crc16_checksum(packet.tobytes()))
    return packet.tobytes()
format_string, data_items = pack_data('autoaim', test_data)
packet = bitstring.pack(format_string, *data_items)
packet

BitStream('0x7b148e3f7b140e40b81e55407b148e409a99b140b81ed540')

In [222]:
def unpack_data(name, packet):
    data = {}
    format_string = ''
    for (key, info) in protocol_info_table[name]['struct'].items():
        new_data = []
        for i in range(info.get('arraysize',1)):
            if 'struct' in info:
                size = protocol_info_table[info['struct']]['size']
                new_data += [unpack_data(info['struct'], packet)]
            else:
                size = math.ceil(int(re.match('.*:(?P<bitsize>\d*)', info['format']).group('bitsize'))/8)
                new_data += bitstring.BitArray(bytes=packet).unpack(info['format'])
            packet = packet[size:]
        if 'arraysize' in info:
            data[key] = new_data
        else:
            data[key] = new_data[0]
    return data
unpack_data('autoaim',packet.tobytes())

{'x': 1.1100000143051147,
 'y': 2.2200000286102295,
 'z': [3.3299999237060547, 4.440000057220459],
 'test': [{'x': 5.550000190734863}, {'x': 6.659999847412109}]}

In [211]:
import re
import math

In [228]:
dict([y for y in [('a',None)] if y[1]])

{}

In [236]:
a=[{'a':1,'b':2},{'a':1,'b':2}]

In [237]:
for (interface_key,interface_id) in [x.items() for x in a]:
    print(interface_key,interface_id)

('a', 1) ('b', 2)
('a', 1) ('b', 2)


In [249]:
test='''
      int32_t data1;
      int32_t data2;
      int32_t data3[3];
      int32_t data4:5;
'''
r = re.compile('\n*?(?P<type>\S*?)\s*?(?P<key>\S*?)(?:\[(?P<arraysize>\d*?)\])?(?:\s?:\s?(?P<bitsize>\d*?))?;')
items = [m.groupdict() for m in r.finditer(test)]
items

[{'type': 'int32_t', 'key': 'data1', 'arraysize': None, 'bitsize': None},
 {'type': 'int32_t', 'key': 'data2', 'arraysize': None, 'bitsize': None},
 {'type': 'int32_t', 'key': 'data3', 'arraysize': '3', 'bitsize': None},
 {'type': 'int32_t', 'key': 'data4', 'arraysize': None, 'bitsize': '5'}]

In [264]:
a=b'\t\x00\x00M\x01\x04\x01{\x14\x8e?{\x14\x0e@\x00&'
try:
    a.index(0xa5)
except ValueError:
    print('...')

...


In [272]:
ccc=bytes([1,2,3])
aaa=ccc
aaa=aaa[1:2]
aaa,ccc

(b'\x02', b'\x01\x02\x03')

In [284]:
aaa=aaa+aaa


In [285]:
aaa

b'\x02\x02\x02\x02\x02\x02\x02\x02'